# Minimal EDA

## make train-test data

In [86]:
!nvidia-smi

Tue May 30 15:02:18 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 525.60.13    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090         Off| 00000000:01:00.0 Off |                  N/A |
| 30%   26C    P8               14W / 350W|      2MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [87]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"# see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"


import torch
torch.cuda.device_count()

1

In [88]:
import pandas as pd

In [89]:
def df_from_file(file, orig_text=None):
    with open('Annotation_1-1-19/'+file, 'r', encoding='UTF-8') as f:
        text = f.read()
        #split on sentence
        sents = text.split('\n\n')
        #split on token
        tokens = [sent.split('\n') for sent in sents]
        tokenstags = [[token.split('\t') for token in sent] for sent in tokens]


    #add sentence number so we can put it in a df
    correct_tags = []    
    for i in range(len(tokenstags)):
        new_sublist = []
        for token in tokenstags[i]:
            try:
                new_sublist.append([token[1], token[0], i])
            except IndexError:
                print(i)
        correct_tags.append(new_sublist)

    flatten = [val for sublist in correct_tags for val in sublist]
    
    df = pd.DataFrame(flatten)
    df.rename(columns={0:'word', 1:'tag',2:'sentence'}, inplace=True)
    if orig_text:
        df['orig_text'] = orig_text
    else:
        df['orig_text'] = file
    return df


In [90]:
CW = df_from_file('CW.crf', 'CW')

257


In [91]:
import pandas as pd

#to (BIO-format)
CW['tag'] = CW['tag'].apply(lambda x: str(x).replace('GEO-B','B-LOC').replace('GEO-I','I-LOC'))
CW['tag'] = CW['tag'].apply(lambda x: str(x).replace('GRP-B','B-GRP').replace('GRP-I','I-GRP'))
CW['tag'] = CW['tag'].apply(lambda x: str(x).replace('PRS-B','B-PERS').replace('PRS-I','I-PERS'))
CW['tag'] = CW['tag'].apply(lambda x: str(x).replace('0', 'O'))



In [92]:
CW['tag'].value_counts()

O         4399
B-PERS     254
B-LOC      113
I-PERS      50
B-GRP       20
Name: tag, dtype: int64

# GW

We do not retain the original train-test split. The train and test sets were split at a 'cutoff point,' so by simply concatenating the train and test data in order, we obtain the complete DBG.

In [93]:
GW = df_from_file('GWtrain.crf', 'GW')

import pandas as pd

#to (BIO-format)
GW['tag'] = GW['tag'].apply(lambda x: str(x).replace('GEO-B','B-LOC').replace('GEO-I','I-LOC'))
GW['tag'] = GW['tag'].apply(lambda x: str(x).replace('GRP-B','B-GRP').replace('GRP-I','I-GRP'))
GW['tag'] = GW['tag'].apply(lambda x: str(x).replace('PRS-B','B-PERS').replace('PRS-I','I-PERS'))
GW['tag'] = GW['tag'].apply(lambda x: str(x).replace('0', 'O'))



GW['tag'].value_counts()

1724


O         37952
B-GRP       953
B-PERS      902
B-LOC       395
I-PERS      174
I-LOC        16
I-GRP         1
Name: tag, dtype: int64

In [94]:
with open('Annotation_1-1-19/GWtest.crf', 'r', encoding='UTF-8') as f:
    text = f.read()
    #split on sentence
    sents = text.split('\n\n')
    #split on token
    tokens = [sent.split('\n') for sent in sents]
    tokenstags = [[token.split('\t') for token in sent] for sent in tokens]


    #add sentence number so we can put it in a df
correct_tags = []    

for i in range(len(tokenstags)):
    new_sublist = []
    for token in tokenstags[i]:
        try:
            new_sublist.append([token[1], token[0], i+len(GW.groupby('sentence'))])
        except IndexError:
            print(i)
    correct_tags.append(new_sublist)

flatten = [val for sublist in correct_tags for val in sublist]


851


In [95]:
GW2 = pd.DataFrame(flatten)

GW2.rename(columns={0:'word', 1:'tag',2:'sentence'}, inplace=True)


GW2['orig_text'] = 'GW'

#to (BIO-format)
GW2['tag'] = GW2['tag'].apply(lambda x: str(x).replace('GEO-B','B-LOC').replace('GEO-I','I-LOC'))
GW2['tag'] = GW2['tag'].apply(lambda x: str(x).replace('GRP-B','B-GRP').replace('GRP-I','I-GRP'))
GW2['tag'] = GW2['tag'].apply(lambda x: str(x).replace('PRS-B','B-PERS').replace('PRS-I','I-PERS'))
GW2['tag'] = GW2['tag'].apply(lambda x: str(x).replace('0', 'O'))



GW2['tag'].value_counts()

O         17517
B-GRP       411
B-PERS      313
B-LOC       203
I-PERS       47
I-GRP         5
I-LOC         2
Name: tag, dtype: int64

In [96]:
GW_all = pd.concat([GW, GW2], ignore_index=True)

# PlinyElder

In [97]:
PE = df_from_file('PlinyElder.crf', 'PlinyElder')

import pandas as pd

#to (BIO-format)
PE['tag'] = PE['tag'].apply(lambda x: str(x).replace('GEO-B','B-LOC').replace('GEO-I','I-LOC'))
PE['tag'] = PE['tag'].apply(lambda x: str(x).replace('GRP-B','B-GRP').replace('GRP-I','I-GRP'))
PE['tag'] = PE['tag'].apply(lambda x: str(x).replace('PRS-B','B-PERS').replace('PRS-I','I-PERS'))
PE['tag'] = PE['tag'].apply(lambda x: str(x).replace('0', 'O'))



PE['tag'].value_counts()

3590


O         32884
B-PERS     1808
B-LOC       468
I-PERS      410
B-GRP       307
I-LOC        30
I-GRP         3
Name: tag, dtype: int64

# PlinyYounger

In [98]:
PY = df_from_file('PlinyYounger.crf', 'PlinyYounger')

import pandas as pd

#to (BIO-format)
PY['tag'] = PY['tag'].apply(lambda x: str(x).replace('GEO-B','B-LOC').replace('GEO-I','I-LOC'))
PY['tag'] = PY['tag'].apply(lambda x: str(x).replace('GRP-B','B-GRP').replace('GRP-I','I-GRP'))
PY['tag'] = PY['tag'].apply(lambda x: str(x).replace('PRS-B','B-PERS').replace('PRS-I','I-PERS'))
PY['tag'] = PY['tag'].apply(lambda x: str(x).replace('0', 'O'))



PY['tag'].value_counts()

1335


O         18058
B-PERS      376
I-PERS      160
B-LOC        47
B-GRP        34
I-LOC         1
Name: tag, dtype: int64

# Ovid

In [99]:
OV = df_from_file('Ovid.crf', 'Ovid')

import pandas as pd

#to (BIO-format)
OV['tag'] = OV['tag'].apply(lambda x: str(x).replace('GEO-B','B-LOC').replace('GEO-I','I-LOC'))
OV['tag'] = OV['tag'].apply(lambda x: str(x).replace('GRP-B','B-GRP').replace('GRP-I','I-GRP'))
OV['tag'] = OV['tag'].apply(lambda x: str(x).replace('PRS-B','B-PERS').replace('PRS-I','I-PERS'))
OV['tag'] = OV['tag'].apply(lambda x: str(x).replace('0', 'O'))



OV['tag'].value_counts()

2434


O         16990
B-PERS      377
B-GRP       107
B-LOC        87
I-PERS        1
Name: tag, dtype: int64

In [100]:
main_df = pd.concat([CW, GW_all, PE, PY, OV])

main_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 135875 entries, 0 to 17561
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   word       135875 non-null  object
 1   tag        135875 non-null  object
 2   sentence   135875 non-null  int64 
 3   orig_text  135875 non-null  object
dtypes: int64(1), object(3)
memory usage: 5.2+ MB


In [101]:
main_df.reset_index(drop=True, inplace=True)

In [102]:
# main_df.to_csv('data/latin_NER_dataset.csv')

In [103]:
# main_df = pd.read_csv('data/latin_NER_dataset.csv', index_col=0)

In [104]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135875 entries, 0 to 135874
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   word       135875 non-null  object
 1   tag        135875 non-null  object
 2   sentence   135875 non-null  int64 
 3   orig_text  135875 non-null  object
dtypes: int64(1), object(3)
memory usage: 4.1+ MB


In [105]:
main_df['tag'].value_counts()

O         127800
B-PERS      4030
B-GRP       1832
B-LOC       1313
I-PERS       842
I-LOC         49
I-GRP          9
Name: tag, dtype: int64

In [106]:
main_df['orig_text'].value_counts()

GW              58891
PlinyElder      35910
PlinyYounger    18676
Ovid            17562
CW               4836
Name: orig_text, dtype: int64

In [107]:
main_df['sent_id'] = main_df['orig_text'] + '_' + main_df['sentence'].astype(str)

In [108]:
main_df.head()

,word,tag,sentence,orig_text,sent_id
0,C.,B-PERS,0,CW,CW_0
1,Iuli,I-PERS,0,CW,CW_0
2,Caesaris,I-PERS,0,CW,CW_0
3,Commentariorum,O,0,CW,CW_0
4,De,O,0,CW,CW_0


In [109]:
def df_sort_groups(df, by='sentence'):
    grouped = df.groupby(by)

    new_df = pd.DataFrame(columns = ['word', 'tag', 'sentence'])

    for name, group in grouped:
        new_df = pd.concat([new_df, group])
        
    return new_df

def df_nested_sort_groups(df, by='orig_text'):
    grouped = df.groupby(by)
    
    new_df = pd.DataFrame(columns = ['word', 'tag', 'sentence'])
    
    for name, group in grouped:
        group = df_sort_groups(group)
        new_df = pd.concat([new_df, group])
        
    return new_df

hel = df_nested_sort_groups(main_df)

In [110]:
hel.tail(40)

,word,tag,sentence,orig_text,sent_id
118273,delitescere,O,1333,PlinyYounger,PlinyYounger_1333
118274,potest,O,1333,PlinyYounger,PlinyYounger_1333
118275,.,O,1333,PlinyYounger,PlinyYounger_1333
118276,Erat,O,1334,PlinyYounger,PlinyYounger_1334
118277,in,O,1334,PlinyYounger,PlinyYounger_1334
118278,consilio,O,1334,PlinyYounger,PlinyYounger_1334
118279,Sertorianum,B-PERS,1334,PlinyYounger,PlinyYounger_1334
118280,illud,O,1334,PlinyYounger,PlinyYounger_1334
118281,exemplum,O,1334,PlinyYounger,PlinyYounger_1334
118282,",",O,1334,PlinyYounger,PlinyYounger_1334


In [111]:
import re
pattern = r'(?!\w+\.)(\W\w+|\w+\W)' #first negative lookahead is to ignore all the letter + point cases which the training script can handle

main_df['special'] = main_df['word'].apply(lambda x: True if re.match(pattern, x) != None else False)

In [112]:
check = main_df[main_df['special'] != False]

In [113]:
check['word'].value_counts()

<COLON>                         1079
δ᾽                                 2
MenelaÉPOSft-lemma=<unknown>       2
?POSft-lemma=<unknown>             1
δ᾽ἕτερον                           1
ἐπ᾽                                1
τ᾽                                 1
γ᾽                                 1
ἤστραπτ᾽                           1
῾ελλάδα                            1
Name: word, dtype: int64

In [114]:
#dropping the above cases from the main_df
main_df.drop(main_df[main_df['special'] == True].index, inplace=True)
main_df.reset_index(drop=True, inplace=True)

In [115]:
main_df['special'].value_counts()

False    134785
Name: special, dtype: int64

In [116]:
main_df.drop(['special'], axis=1, inplace=True)

In [117]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134785 entries, 0 to 134784
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   word       134785 non-null  object
 1   tag        134785 non-null  object
 2   sentence   134785 non-null  int64 
 3   orig_text  134785 non-null  object
 4   sent_id    134785 non-null  object
dtypes: int64(1), object(4)
memory usage: 5.1+ MB


In [118]:
main_df.to_csv('data/latin_NER_dataset_improved.csv')

## check ents

In [119]:
GRP = main_df[main_df['tag'].str.endswith('GRP')]

In [120]:
GRP.word.value_counts()[:40]

Romani         84
Gallorum       36
Romanis        32
Romanorum      31
Germani        31
Romanos        29
Galli          27
Gallis         26
Germanorum     23
Helvetii       23
Germanos       23
Aeduorum       22
Romano         22
Germanis       19
Aeduis         18
Aeduos         16
Gallos         16
Helvetiorum    15
Aedui          15
Graeci         14
Carnutes       14
Helvetios      14
Romanum        14
Sequanos       13
Haeduos        13
Arvernis       13
Haeduorum      13
Treverorum     13
Remis          12
Sequanis       12
Helvetiis      12
Graece         12
Haeduis        12
Bellovacos     11
Belgis         10
Bituriges      10
Remorum        10
Treveros       10
Biturigum       9
Haedui          9
Name: word, dtype: int64